# SQL Assignment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [ ]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [ ]:
conn = sqlite3.connect("/content/drive/MyDrive/assgn 18 SQL on imdb/Db-IMDB-Assignment.db")
cursor = conn.cursor()
print('successfull')

successfull


#### Overview of all tables

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
tables

['Movie',
 'Genre',
 'Language',
 'Country',
 'Location',
 'M_Location',
 'M_Country',
 'M_Language',
 'M_Genre',
 'Person',
 'M_Producer',
 'M_Director',
 'M_Cast']

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [ ]:
query1 = """
SELECT DISTINCT TRIM(Name),TRIM(title),TRIM(year) FROM Movie M JOIN M_Director D ON M.MID = D.MID
JOIN Person P ON P.PID = D.PID
JOIN (SELECT MID,GID FROM M_Genre WHERE GID IN (SELECT GID FROM Genre WHERE Name LIKE '%Comedy%')) AS G
ON G.MID = M.MID
WHERE (CAST(year AS int) % 400) OR (CAST(year AS int) % 4 AND NOT CAST(year AS int) % 100) ;
"""
for row in cursor.execute(query1):
    print(row)

('Gary Ross', "Ocean's Eight", '2018')
('Sriram Raghavan', 'Andhadhun', '2018')
('Leena Yadav', 'Rajma Chawal', '2018')
('Milap Zaveri', 'Mastizaade', '2016')
('Chris Columbus', 'Pixels', '2015')
('Amar Kaushik', 'Stree', '2018')
('Amit Ravindernath Sharma', 'Badhaai Ho', '2018')
('Wes Anderson', 'The Darjeeling Limited', '2007')
('Gurinder Chadha', 'Bend It Like Beckham', '2002')
('Vishal Bhardwaj', 'Pataakha', '2018')
('Danny Leiner', 'Harold & Kumar Go to White Castle', '2004')
('Rajkumar Hirani', '3 Idiots', '2009')
('Rajkumar Hirani', 'PK', '2014')
('Lasse Hallström', 'The Hundred-Foot Journey', '2014')
('Anurag Kashyap', 'Gangs of Wasseypur', '2012')
('Akarsh Khurana', 'Karwaan', '2018')
('Nitin Kakkar', 'Mitron', '2018')
('R. Balki', 'Padman', '2018')
('John Madden', 'The Best Exotic Marigold Hotel', '2011')
('Frank Coraci', 'Around the World in 80 Days', '2004')
('Ashwiny Iyer Tiwari', 'Bareilly Ki Barfi', '2017')
('Rakeysh Omprakash Mehra', 'Rang De Basanti', '2006')
('Anand T

## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
sql_query2 = """
SELECT DISTINCT TRIM(Name) FROM Person WHERE PID IN (
SELECT DISTINCT TRIM(PID) FROM M_Cast WHERE MID = (
SELECT TRIM(MID) FROM Movie M WHERE TRIM(M.title) = 'Anand')) ;
"""
for row in cursor.execute(sql_query2):
    print(row)

('Amitabh Bachchan',)
('Rajesh Khanna',)
('Sumita Sanyal',)
('Ramesh Deo',)
('Seema Deo',)
('Asit Kumar Sen',)
('Dev Kishan',)
('Atam Prakash',)
('Lalita Kumari',)
('Savita',)
('Brahm Bhardwaj',)
('Gurnam Singh',)
('Lalita Pawar',)
('Durga Khote',)
('Dara Singh',)
('Johnny Walker',)
('Moolchand',)


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
query3= """
SELECT DISTINCT TRIM(Name) FROM Person WHERE PID IN (
SELECT DISTINCT TRIM(PID) FROM M_Cast WHERE MID IN (
SELECT TRIM(MID) FROM Movie WHERE year<1970 )
INTERSECT
SELECT DISTINCT TRIM(PID) FROM M_Cast WHERE MID IN (
SELECT TRIM(MID) FROM Movie WHERE year>1990 )
)
"""
for row in cursor.execute(query3):
    print(row)

('Rishi Kapoor',)
('Amitabh Bachchan',)
('Asrani',)
('Zohra Sehgal',)
('Parikshat Sahni',)
('Rakesh Sharma',)
('Sanjay Dutt',)
('Ric Young',)
('Yusuf',)
('Suhasini Mulay',)
('A.K. Hangal',)
('Jeremy Child',)
('Farida Jalal',)
('Waheeda Rehman',)
('Rajesh Khanna',)
('Ramesh Deo',)
('Seema Deo',)
('Asit Kumar Sen',)
('Lalita Kumari',)
('Brahm Bhardwaj',)
('Lalita Pawar',)
('Dara Singh',)
('Johnny Walker',)
('Moolchand',)
('Saira Banu',)
('Prem Chopra',)
('Dina Pathak',)
('Achala Sachdev',)
('Shashikala',)
('Mohandas K. Gandhi',)
('Jawaharlal Nehru',)
('Naseeruddin Shah',)
('Usha Kiran',)
('Zul Vellani',)
('Rekha',)
('Shubha Khote',)
('Surendra Rahi',)
('Shammi Kapoor',)
('Bindu',)
('Dharmendra',)
('Sunil Dutt',)
('Subhash Ghai',)
('Jeetendra',)
('Feroz Khan',)
('Sitaram',)
('Hema Malini',)
('Poonam Sinha',)
('Sachin',)
('Sarika',)
('Neetu Singh',)
('Sanjeev Kumar',)
('Amjad Khan',)
('Satyendra Kapoor',)
('Iftekhar',)
('Leela Mishra',)
('Mac Mohan',)
('Keshto Mukherjee',)
('Viju Khote',)


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
query4 = """
SELECT DISTINCT TRIM(P.Name), MC.M_Count FROM Person P JOIN (
SELECT TRIM(PID) As PID,COUNT(MID) AS M_Count FROM M_Director
GROUP BY PID HAVING COUNT(MID)>=10 ) AS MC ON MC.PID = P.PID
ORDER BY MC.M_Count DESC
"""
for row in cursor.execute(query4):
    print(row)

('David Dhawan', 39)
('Mahesh Bhatt', 35)
('Priyadarshan', 30)
('Ram Gopal Varma', 30)
('Vikram Bhatt', 29)
('Hrishikesh Mukherjee', 27)
('Yash Chopra', 21)
('Basu Chatterjee', 19)
('Shakti Samanta', 19)
('Subhash Ghai', 18)
('Shyam Benegal', 17)
('Abbas Alibhai Burmawalla', 17)
('Rama Rao Tatineni', 17)
('Manmohan Desai', 16)
('Gulzar', 16)
('Raj N. Sippy', 16)
('Raj Kanwar', 15)
('Mahesh Manjrekar', 15)
('Indra Kumar', 14)
('Raj Khosla', 14)
('Rahul Rawail', 14)
('Rajkumar Santoshi', 14)
('Rakesh Roshan', 13)
('Dev Anand', 13)
('Vijay Anand', 13)
('Harry Baweja', 13)
('Anurag Kashyap', 13)
('Ananth Narayan Mahadevan', 13)
('K. Raghavendra Rao', 13)
('Anees Bazmee', 12)
('Guddu Dhanoa', 12)
('Prakash Jha', 12)
('Satish Kaushik', 12)
('Nagesh Kukunoor', 12)
('Prakash Mehra', 12)
('Umesh Mehra', 12)
('Anil Sharma', 12)
('Madhur Bhandarkar', 12)
('Rohit Shetty', 12)
('Pramod Chakravorty', 11)
('Sanjay Gupta', 11)
('Nasir Hussain', 11)
('Ketan Mehta', 11)
('Govind Nihalani', 11)
('Mohit S

## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
query5a = """
SELECT year,COUNT(MID) FROM Movie WHERE TRIM(MID) NOT IN (
SELECT DISTINCT TRIM(C.MID) FROM M_Cast C JOIN Person P ON TRIM(C.PID)=TRIM(P.PID)
WHERE TRIM(P.Gender) = 'Male') GROUP BY year;
"""
for row in cursor.execute(query5a):
    print(row)

('1939', 1)
('1999', 1)
('2000', 1)
('I 2018', 1)


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [ ]:
query5b = """
SELECT FMovies.year,FMovies.Count,(FMovies.Count*100.0)/COUNT(TRIM(MID)) FROM Movie M JOIN (
SELECT year,COUNT(TRIM(MID)) as Count FROM Movie WHERE TRIM(MID) NOT IN (
SELECT DISTINCT TRIM(C.MID) FROM M_Cast C JOIN Person P ON TRIM(C.PID)=TRIM(P.PID)
WHERE TRIM(P.Gender) = 'Male') GROUP BY year  ) AS FMovies ON M.year=FMovies.year
GROUP BY FMovies.year
"""
for row in cursor.execute(query5b):
    print(row)

('1939', 1, 50.0)
('1999', 1, 1.5151515151515151)
('2000', 1, 1.5625)
('I 2018', 1, 10.0)


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [ ]:
query6 = """
SELECT title,COUNT(DISTINCT PID) AS Ncast FROM Movie M JOIN M_Cast C ON
TRIM(M.MID)=TRIM(C.MID)
GROUP BY M.MID HAVING Ncast = ( SELECT MAX(NC.PCount) FROM
                       (SELECT COUNT(DISTINCT PID) AS PCount
                        FROM M_Cast GROUP BY MID) NC)
"""
for row in cursor.execute(query6):
    print(row)

("Ocean's Eight", 238)


### Q7 --- A decade is a sequence of 10 consecutive years.
### For example, say in your database you have movie information starting from 1931.
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on.
### Find the decade D with the largest number of films and the total number of films in D

In [ ]:
query7 = """
SELECT year-year%10 As Decade, COUNT(DISTINCT MID) as NumM
FROM Movie
WHERE LENGTH(year)=4
GROUP BY Decade
ORDER BY NumM DESC LIMIT 1
"""
for row in cursor.execute(query7):
    print(row)

(2010, 1017)


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
query8 = """
WITH Cast_Fav_Dir AS (
SELECT CID, DID, Num_Movies,
ROW_NUMBER() OVER( PARTITION BY CID ORDER BY Num_Movies DESC) Row_Num FROM (
SELECT TRIM(C.PID) AS CID,TRIM(D.PID) AS DID,COUNT(DISTINCT TRIM(C.MID)) AS Num_Movies
FROM M_Cast C JOIN M_Director D ON TRIM(C.MID)=TRIM(D.MID)
GROUP BY TRIM(C.PID),TRIM(D.PID)) AS TEMP     )

SELECT DISTINCT TRIM(Name) FROM Person WHERE PID IN (
SELECT DISTINCT CID FROM Cast_Fav_Dir AS FD WHERE Row_Num = 1
AND DID IN (SELECT DISTINCT TRIM(PID) FROM Person WHERE NAME LIKE '%YASH%CHOPRA%' ))
"""
for row in cursor.execute(query8):
    print(row)

('Kulbir Badesron',)
('Gurdas Maan',)
('Parikshat Sahni',)
('Waheeda Rehman',)
('Taj Gill',)
('Kumud Pant',)
('Gerald Tomkinson',)
('Dev K. Kantawall',)
('Harish Chandra',)
('Achala Sachdev',)
('Darshan Aulakh',)
('Kanwar Jagdish',)
('Sharan Hunjan',)
('Dolly Jagdeo',)
('Vinita Sharma',)
('Steven Baker',)
('Andrew Bicknell',)
('Banwarhlal Jhol',)
('Kimti Anand',)
('Damyanti Puri',)
('Hemlata Deepak',)
('Surendra Rahi',)
('Yash Chopra',)
('Vinod Negi',)
('Balwant Bansal',)
('Rajesh Jolly',)
('Anup Kanwal Singh',)
('Ranjeev Verma',)
('Chandni Jas Keerat',)
('Sanjay Singh Bhadli',)
('Shivaya Singh',)
('Huzefa Gadiwala',)
('Amir Zadey',)
('Syed Firdaus',)
('Manish Arora',)
('Jitendra Bhardwaj',)
('Pankaj Raina',)
('Gurinder Singh',)
('Mike Parish',)
('Tom Swacha',)
('Jon Wennington',)
('Neetu Singh',)
('Julia St John',)
('Susan Fordham',)
('Lee Nicholas Harris',)
('Vic Waghorn',)
('Steve Box',)
('Paul Blackwell',)
('Jasmine Jardot',)
('Abbie Murphy',)
('Julie Vollono',)
('Varun Thakur',)
(

## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
query9 = """
SELECT DISTINCT TRIM(Name) FROM Person WHERE PID IN (
SELECT DISTINCT TRIM(PID) AS PID FROM M_Cast WHERE TRIM(MID) IN (
SELECT DISTINCT TRIM(MID) FROM M_Cast WHERE TRIM(PID) IN (
SELECT DISTINCT TRIM(PID) FROM M_Cast WHERE TRIM(MID) IN (
SELECT DISTINCT TRIM(MID) FROM M_Cast WHERE TRIM(PID) = (
SELECT TRIM(PID) FROM Person WHERE Name LIKE '%Shah Rukh Khan%'))))
EXCEPT
SELECT DISTINCT TRIM(PID) FROM M_Cast WHERE TRIM(MID) IN (
SELECT DISTINCT TRIM(MID) FROM M_Cast WHERE TRIM(PID) = (
SELECT TRIM(PID) FROM Person WHERE Name LIKE '%Shah Rukh Khan%')) )
"""
for row in cursor.execute(query9):
    print(row)

Streaming output truncated to the last 5000 lines.
('Master Aayush Kapssey',)
('Chhavi Mittal',)
('Amit Patil',)
('Master Alok Srivastav',)
('Ritu Vasishtha',)
('Alummoodan',)
('Vettoor Purushan',)
('Sonia',)
('Santosh P. Jadhav',)
('Mallika Bhat',)
('Balu',)
('Joyce Arora',)
('Ajay Bhagwat',)
('Master Smit Parekh',)
('Laxman Gollar',)
('Vallabhbhai Patel',)
('Vishal Dutt',)
('Ravindra Thakur',)
('Butty',)
('Jamini',)
('Pragna',)
('Shalini',)
('Satyen',)
('Tora Khasgir',)
('Gaurav Rai',)
('Narra Venkateswara Rao',)
('Chitram Seenu',)
('Vinita Dutt',)
('Shaminder',)
('Kharaiti',)
('Jani',)
('Harbans',)
('Surendra Shailaj',)
('Chaman',)
('Roshanlal',)
('Beshir',)
('Tarsem',)
('Masood',)
('Baijnath',)
('Kamal Dukhia',)
('Rajendra Agarwal',)
('Chandra Mohan Sahay',)
('V.P. Singh',)
('V.P. Shinde',)
('Lali',)
('Mohammad Aslam',)
('Daman Maan',)
('Sonarika Bhadoria',)
('Amir Dalvi',)
('Neetha Shetty',)
('Aradhana Jagota',)
('Karan Mahavar',)
('Reecha Sharma',)
('Talat Rekhi',)
('Sophie Handa